<a href="https://colab.research.google.com/github/ashu131998/Webapp/blob/master/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

!pip install tensorflow==1.14
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
!pip install keras==2.3.1
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.common.set_image_dim_ordering('th')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [2]:
from io import BytesIO

from tensorflow.python.lib.io import file_io

In [3]:
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/ceiling fan.npy', binary_mode=True))
fan = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/cat.npy', binary_mode=True))
cat = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/camel.npy', binary_mode=True))
camel = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/bicycle.npy', binary_mode=True))
bicycle = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/camera.npy', binary_mode=True))
camera = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/light bulb.npy', binary_mode=True))
bulb = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/hand.npy', binary_mode=True))
hand = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/car.npy', binary_mode=True))
eyeglasses = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/ladder.npy', binary_mode=True))
ladder = np.load(f)
f = BytesIO(file_io.read_file_to_string('gs://quickdraw_dataset/full/numpy_bitmap/axe.npy', binary_mode=True))
axe = np.load(f)



In [4]:
fan = np.c_[fan, 1*np.ones(len(fan))]
cat = np.c_[cat, 3*np.ones(len(cat))]
camel = np.c_[camel, 5*np.ones(len(camel))]
bicycle= np.c_[bicycle, 7*np.ones(len(bicycle))]
camera= np.c_[camera, 8*np.ones(len(camera))]
bulb=np.c_[bulb, 9*np.ones(len(bulb))]
hand=np.c_[hand, 10*np.ones(len(hand))]
eyeglasses=np.c_[eyeglasses, 12*np.ones(len(eyeglasses))]
ladder=np.c_[ladder, 13*np.ones(len(ladder))]

In [5]:
X = np.concatenate((fan[:10000, :-1],cat[:10000,:-1], camel[:10000, :-1],bicycle[:10000, :-1],camera[:10000,:-1], bulb[:10000,:-1], hand[:10000, :-1], ladder[:10000, :-1]), axis=0).astype('float32') # all columns but the last
y = np.concatenate((fan[:10000,-1], cat[:10000,-1], camel[:10000,-1],bicycle[:10000,-1],camera[:10000,-1],bulb[:10000,-1],hand[:10000,-1],ladder[:10000,-1]), axis=0).astype('float32') # the last column

# We than split data between train and test (80 - 20 usual ratio). Normalizing the value between 0 and 1
X_train, X_test, y_train, y_test = train_test_split(X/255.,y,test_size=0.2,random_state=0)

In [6]:
y_train_cnn = np_utils.to_categorical(y_train)
y_test_cnn = np_utils.to_categorical(y_test)
num_classes = y_test_cnn.shape[1]

In [7]:
X_train_cnn = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test_cnn = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [8]:
def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (3, 3), input_shape=(1, 28, 28), activation='relu',data_format='channels_first'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
np.random.seed(0)
# build the model
model_cnn = cnn_model()
# Fit the model
model_cnn.fit(X_train_cnn, y_train_cnn, validation_data=(X_test_cnn, y_test_cnn), epochs=20, batch_size=200)
# Final evaluation of the model
scores = model_cnn.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print('Final CNN accuracy: ', scores[1])



Train on 64000 samples, validate on 16000 samples
Epoch 1/20
64000/64000 [==============================] - 52s 818us/step - loss: 0.8556 - accuracy: 0.7227 - val_loss: 0.4783 - val_accuracy: 0.8543
Epoch 2/20
64000/64000 [==============================] - 52s 810us/step - loss: 0.4323 - accuracy: 0.8696 - val_loss: 0.3250 - val_accuracy: 0.9036
Epoch 3/20
64000/64000 [==============================] - 52s 808us/step - loss: 0.3393 - accuracy: 0.8991 - val_loss: 0.2837 - val_accuracy: 0.9149
Epoch 4/20
64000/64000 [==============================] - 52s 810us/step - loss: 0.2946 - accuracy: 0.9134 - val_loss: 0.2468 - val_accuracy: 0.9264
Epoch 5/20
64000/64000 [==============================] - 52s 808us/step - loss: 0.2644 - accuracy: 0.9216 - val_loss: 0.2349 - val_accuracy: 0.9293
Epoch 6/20
64000/64000 [==============================] - 52s 807us/step - loss: 0.2412 - accuracy: 0.9268 - val_loss: 0.2101 - val_accuracy: 0.9376
Epoch 7/20
64000/64000 [==============================

In [10]:
model_cnn.save("model.h5")

In [11]:
!pip install tensorflowjs

     |████████████████████████████████| 61kB 3.1MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 137.3MB 126kB/s 
     |████████████████████████████████| 6.8MB 39.9MB/s 
     |████████████████████████████████| 460kB 42.9MB/s 
     |████████████████████████████████| 256kB 46.5MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32852 sha256=c37ac5aecc35e027f6a371f9295e6b9a49e2ac46eff07c087b986bbd5e9845db
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 2.3.0 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 2.3.0 which is incompatible.
  Found existing installation: tensorflow-hub 0.9.0
    Uninstalling tensorflow-hub-0.9.0:
  

In [13]:
!tensorflowjs_converter --input_format keras '/content/model.h5' '/content/model'